# Detección de Objectos

En este tutorial veremos como se puede utilizar Pytorch para entrenar un modelo de detección
de objetos.

Usaremos la base de datos [*Penn-Fudan Database for Pedestrian Detection and Segmentation*](https://www.cis.upenn.edu/~jshi/ped_html/)
y la entrenaremos utilizando un modelo de [Mask R-CNN](https://arxiv.org/abs/1703.06870). Este modelu incluye un total de
170 imágenes con 345 instancias de peatones, y lo usaremos para ver como podemos extraer "features" para entrenar un modelo

In [ ]:
# Instalamos los paquetes necesarios
!pip install -r https://raw.githubusercontent.com/vcalderon2009/2021_06_Deep_Learning_tutorial/feature/refactoring/pkg_requirements.txt -q

In [ ]:
# Importing modules
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import shutil

## Dataset

Primero bajamos la base de datos en la carpeta actual:

In [ ]:
# download the Penn-Fudan dataset
!wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip .
# extract it in the current folder
!unzip PennFudanPed.zip

Ahora podemos ver la estructura de esta nueva carpeta:

```
PennFudanPed/
  PedMasks/
    FudanPed00001_mask.png
    FudanPed00002_mask.png
    FudanPed00003_mask.png
    FudanPed00004_mask.png
    ...
  PNGImages/
    FudanPed00001.png
    FudanPed00002.png
    FudanPed00003.png
    FudanPed00004.png
```

Cada una de estas imágenes tiene asociado las máscaras que enseñan en donde es que hay personas.

Por ejemplo, podemos ver una de estas imágenes:

In [ ]:
from PIL import Image

# Número de imagen
image_number = "00003"

Image.open(f'PennFudanPed/PNGImages/FudanPed{image_number}.png')

Y también podemos ver la **máscara** asociada a esta imagen:

In [ ]:
mask = Image.open(f'PennFudanPed/PedMasks/FudanPed{image_number}_mask.png')
# cada instancia de máscara tiene un color diferente, de cero a N, donde
# N es el número de instancias. Para facilitar la visualización,
# agreguemos paleta de colores a la máscara.
mask.putpalette([
    0, 0, 0, # black background
    255, 0, 0, # index 1 is red
    255, 255, 0, # index 2 is yellow
    255, 153, 0, # index 3 is orange
])
mask

Esto significal que cada imagen tiene una máscara de segmentación asociada, en donde el color corresponde a las
diferentes instancias.

## Dataloader

La siguiente parte es el **Dataset**, en donde se define como leer y ejecutar la data:

In [ ]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # Lee todas las imágenes y las ordena para ver que estén todas alineadas
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # Leyendo las imágenes y máscaras
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # NOTE: tenga en cuenta que no hemos convertido la máscara a RGB,
        # porque cada color corresponde a una instancia diferente
        # siendo 0 el fondo
        mask = Image.open(mask_path)

        mask = np.array(mask)
        # las instancias están codificadas como colores diferentes
        obj_ids = np.unique(mask)
        # la primera identificación es el fondo, así que elimínela
        obj_ids = obj_ids[1:]

        # dividir la máscara codificada por colores en un conjunto
        # de máscaras binarias
        masks = mask == obj_ids[:, None, None]

        # obtener las coordenadas del cuadro delimitador para cada máscara
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Solamente hay una clase para acá
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # supongamos que no todas las instancias son multitud
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

Ahora que ya hemos definido el objecto del `Dataset`, podemos ver como se estructura los objetos:

In [ ]:
dataset = PennFudanDataset('PennFudanPed/')
dataset[100]

Este objecto devuelve lo siguite:
- La imágen en formato de `PIL.image`.
- Un diccionario con el metadata de `boxes`, `labels`, y `masks`. Toda esta información se puede utilizar para entrenar un modelo.

### Modelaje

En este tutorial, usaremos [Mask R-CNN](https://arxiv.org/abs/1703.06870), el cual está basado en la red neural [Faster R-CNN](https://arxiv.org/abs/1506.01497). Faster R-CNN
es un modelo que predice tanto cuadros delimitadores como puntuaciones de clase para objetos potenciales en la imagen.

![Faster R-CNN](https://raw.githubusercontent.com/pytorch/vision/temp-tutorial/tutorials/tv_image03.png)

Mask R-CNN agrega una rama adicional a Faster R-CNN, que también predice máscaras de segmentación para cada instancia.

![Mask R-CNN](https://raw.githubusercontent.com/pytorch/vision/temp-tutorial/tutorials/tv_image04.png)

##### Entrenando el modelo

Para este tutorial usaremos un modelo que ya ha sido entrenado previamente, y utilizaremos las pesas
de este modelo para aplicarlo a nuestra base de datos:

In [ ]:
# Importamos los módulos necesarios, como los modelos y demás
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # En este caso, estamos cargando ya un modelo entrenado de RESNET sobre la
    # base de COCO.
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # Obtenemos el número de características de entrada para el clasificador
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # Reemplazamos la cabeza previamente entrenada por una nueva.
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Ahora podemos obtener el número de características de entrada para el clasificador de máscara
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

Ahora necesitamos copiar unos archivos de `Pytorch` para usarlos acá:

In [ ]:
!git clone -b v0.3.0 --single-branch https://github.com/pytorch/vision.git

In [ ]:
# Now we can copy files from the following location
for file_ii in ["utils.py", "transforms.py", "coco_eval.py", "engine.py", "coco_utils.py"]:
    shutil.copy(f"./vision/references/detection/{file_ii}", ".")

Y ahora podemos escribir las funciones para transformar las imágenes:

In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

Ahora creamos el dataset total:

In [ ]:
# use our dataset and defined transformations
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

Y empezamos el entrenamiento del modelo:

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Iteramos sobre el diferente número de épocas:

In [ ]:
# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

#### Evaluciación del modelo

Ahora podemos ver que tan bien aprendió el modelo:

In [ ]:
# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
# Y esta es la máscara
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())